Reto: Modelo basado en deep learning para la discriminación entre gatos y perros

Act:DS_C7SC4

Eduardo Frias Rosales

Introducción:

Han pasado más de dos años desde que se descubrió la vacuna contra los zombis. Ahora un nuevo peligro amenaza el mundo. Algunas razas de perros son inmunes a la vacuna y pueden crear una nueva cepa del virus. La empresa estadounidense Small Pet, una vez que se dio cuenta del problema, dio aviso a todos los países para que estén alerta.

En México, la empresa Ciencia para el futuro se ha propuesto apoyar con la creación de una aplicación para identificar a dichos perros. Por ello, te ha contratado para que crees una App antes de que los perros infecten a su primera víctima con la nueva cepa del virus mortal.

¿Podrás lograrlo antes de que el mundo caiga nuevamente en manos de los zombis?

Objetivos:

Diseñar modelos de redes neuronales profundas (deep learning), enfocándose en la clasificación de imágenes de acuerdo a categorías preestablecidas para resolver problemas con relevancia social, permitiendo generar valor en los diversos sectores.
Crear modelos de datos Deep Neural Networks (DNN) utilizando PyTorch basado en Python; seleccionando el modelo adecuado y analizando la exactitud del modelo, para cumplir lo mejor posible con los requerimientos de la tarea requerida.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
import torchvision.models as models
from PIL import Image
import os

# Definir la ruta a los datos
data_dir = "/content/drive/MyDrive/Colab Notebooks/data/Deep learning utilizando PyTorch/catsvsdogs"

# Lista las imágenes en el directorio
image_files = []
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith(('.jpg', '.jpeg', '.png')):
            image_files.append(os.path.join(root, file))

print(f"Número de imágenes encontradas: {len(image_files)}")

Número de imágenes encontradas: 4895


In [2]:
# Implementar un clasificador que permita discriminar entre perros y gatos. En la libreta implementa tu modelo y realiza las pruebas que consideres pertinente.

# Definir las transformaciones para el conjunto de datos
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Crear el conjunto de datos
dataset = datasets.ImageFolder(data_dir, transform=transform)

# Dividir el conjunto de datos en entrenamiento y validación
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Crear los cargadores de datos
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Definir el modelo
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 clases: gatos y perros

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenar el modelo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

    # Validar el modelo
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy on validation set: {100 * correct / total}%")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 60.4MB/s]


Epoch 1/10, Loss: 0.21286801762152008
Accuracy on validation set: 94.68845760980592%
Epoch 2/10, Loss: 0.1302354169169032
Accuracy on validation set: 93.97344228804903%
Epoch 3/10, Loss: 0.0934662316164865
Accuracy on validation set: 93.97344228804903%
Epoch 4/10, Loss: 0.06721049732702776
Accuracy on validation set: 92.64555669050051%
Epoch 5/10, Loss: 0.053819253845746075
Accuracy on validation set: 90.39836567926456%
Epoch 6/10, Loss: 0.0727322124402849
Accuracy on validation set: 93.25842696629213%
Epoch 7/10, Loss: 0.0614357177221133
Accuracy on validation set: 92.03268641470889%
Epoch 8/10, Loss: 0.058988009403437014
Accuracy on validation set: 95.09703779366701%
Epoch 9/10, Loss: 0.034744454172489087
Accuracy on validation set: 89.17262512768131%
Epoch 10/10, Loss: 0.036575541674676235
Accuracy on validation set: 93.97344228804903%


In [3]:
# Guardar el modelo entrenado en Google Drive
model_path = "/content/drive/MyDrive/Colab Notebooks/data/Deep learning utilizando PyTorch/cats_vs_dogs_model.pth"
torch.save(model.state_dict(), model_path)
